In [50]:
from PIL import Image
from io import BytesIO

In [51]:
# 尝试导入 Pillow，如果没装，就定义一个假的 Image 对象，保证代码不崩溃
try:
    from PIL import Image
    HAS_PILLOW = True
except ImportError:
    print("⚠️ 警告: 没有安装 Pillow 库 (pip install Pillow)。图片将不会被压缩，文件体积可能很大。")
    HAS_PILLOW = False


In [69]:
import folium
from folium.features import DivIcon
import base64
import os

In [74]:
# ================= 1. 配置区域 =================

# 图片文件夹路径
IMAGE_FOLDER = "pictures"

# 地点数据列表 (双语 + 详细描述 + 视频链接)
locations = [
    {
        "name_cn": "亭林园顾炎武纪念馆", 
        "name_en": "Gu Yanwu Memorial Hall",
        "lat": 31.393724, "lon": 120.947496,
        "type_cn": "博物馆", "type_en": "Museum",
        "desc_cn": "纪念明末清初著名思想家顾炎武先生的专题纪念馆。馆内陈列其生平事迹、著作及学术成就，弘扬'天下兴亡，匹夫有责'的爱国主义精神。",
        "desc_en": "A memorial hall dedicated to Gu Yanwu, a famous scholar of the late Ming and early Qing dynasties. It displays his life story and academic achievements, highlighting his famous quote: 'Everyone is responsible for the rise and fall of the world'.",
        "img": "guyanwu_museum.jpg",
    },
    {
        "name_cn": "昆曲博物馆", 
        "name_en": "Kunqu Opera Museum",
        "lat": 31.392813, "lon":120.947891,
        "type_cn": "文化", "type_en": "Culture",
        "desc_cn": "展示'百戏之祖'昆曲艺术的殿堂。通过戏台、服饰、乐器等展品，全方位呈现昆曲的历史源流与艺术魅力。",
        "desc_en": "A museum showcasing the art of Kunqu Opera, known as the 'Ancestor of All Dramas'. It presents the history and charm of Kunqu through stages, costumes, and instruments.",
        "img": "kunqu.jpg",
    },
    {
        "name_cn": "机器人饮品站", 
        "name_en": "Robot Drink Station",
        "lat": 31.392641, "lon": 120.947228,
        "type_cn": "服务", "type_en": "Service",
        "desc_cn": "亭林园内的现代科技体验点，机械臂现场制作饮品，将传统园林与现代科技完美融合。",
        "desc_en": "A modern tech experience spot in Tinglin Garden, where robotic arms prepare drinks on-site, blending traditional gardens with modern technology.",
        "img": "robot_drink.jpg",
        "video_link": "detail_robot.html"
    },
    {
        "name_cn": "祖冲之像", 
        "name_en": "Statue of Zu Chongzhi",
        "lat": 31.392113, "lon": 120.946239,
        "type_cn": "雕像", "type_en": "Statue",
        "desc_cn": "纪念曾在昆山任县令的古代伟大数学家、天文学家祖冲之。他将圆周率精确到小数点后七位。",
        "desc_en": "A statue commemorating Zu Chongzhi, the great ancient mathematician and astronomer who served as a magistrate in Kunshan. He calculated Pi to seven decimal places.",
        "img": "zuchongzhi.jpg",
    },

      {
        "name_cn": "昆石馆",
        "name_en": "Kun Stone Gallery",
        "lat": 31.394680, "lon": 120.948121,
        "type_cn": "展馆",
        "type_en": "Gallery",
        "desc_cn": "展示昆石与园林石文化的空间。",
        "desc_en": "A gallery dedicated to Kun stones and traditional garden rock culture.",
        "img": "kunstone.jpg",
       
    },

    {
        "name_cn": "留晖亭", "name_en": "Liuhui Pavilion",
        "lat": 31.389771, "lon": 120.941559,
        "type_cn": "亭台", "type_en": "Pavilion",
        "desc_cn": "亭依山而建，位置绝佳，可在此欣赏朝晖夕阴之美，是休憩观景的好去处。",
        "desc_en": "Built against the mountain, this pavilion offers a perfect spot to enjoy the beauty of sunrise and sunset.",
        "img": "liuhui.jpg"
    },
    {
        "name_cn": "问石苑", "name_en": "Wenshi Garden",
        "lat": 31.390062, "lon": 120.941870,
        "type_cn": "园林", "type_en": "Garden",
        "desc_cn": "园中奇石林立，汇集了众多形态各异的昆石，展现了昆山深厚的石文化底蕴。",
        "desc_en": "Filled with unique rock formations, this garden showcases the profound stone culture of Kunshan through various Kunstones.",
        "img": "wenshi.jpg"
    },
    {
        "name_cn": "琼华阁", "name_en": "Qionghua Pavilion",
        "lat": 31.391128, "lon": 120.940179,
        "type_cn": "建筑", "type_en": "Building",
        "desc_cn": "因琼花而得名，建筑古朴典雅。每当春日来临，琼花盛开，洁白如玉，蔚为壮观。",
        "desc_en": "Named after the Jade Flower (Viburnum), this elegant building is surrounded by spectacular white blooms in spring.",
        "img": "qionghua.jpg"
    },
    {
        "name_cn": "并蒂莲池", "name_en": "Twin Lotus Pond",
        "lat": 31.394367, "lon": 120.948376,
        "type_cn": "水景", "type_en": "Pond",
        "desc_cn": "池中种植着珍稀的并蒂莲，象征着吉祥喜庆，是夏日赏荷的绝佳地点。",
        "desc_en": "Home to the rare Twin Lotus, symbolizing good fortune and happiness. An excellent spot for viewing lotuses in summer.",
        "img": "lotus_pond.jpg"
    },
    {
        "name_cn": "玉峰门", "name_en": "Yufeng Gate",
        "lat": 31.393350, "lon": 120.947234,
        "type_cn": "大门", "type_en": "Gate",
        "desc_cn": "亭林园的主要入口之一，气势恢宏，上方匾额题字苍劲有力。",
        "desc_en": "One of the main entrances to Tinglin Garden, featuring magnificent architecture and powerful calligraphy on the plaque.",
        "img": "yufeng_gate.jpg"
    },
    {
        "name_cn": "千年银杏", "name_en": "Millennium Ginkgo",
        "lat": 31.393331, "lon": 120.947115,
        "type_cn": "古树", "type_en": "Ancient Tree",
        "desc_cn": "这株银杏树历经千年风雨，依然枝繁叶茂。深秋时节，金叶满地，美不胜收。",
        "desc_en": "Verify weathering a thousand years, this ginkgo tree remains lush. In late autumn, its golden leaves create a breathtaking scene.",
        "img": "ginkgo.jpg"
    },
    
    
    {"name_cn": "岁寒亭", "name_en": "Suihan Pavilion", 
     "lat": 31.390355, "lon": 120.941198,
     "type_cn": "亭台", "type_en": "Pavilion", 
     "desc_cn": "取意‘岁寒，然后知松柏之后凋也’。", 
     "desc_en": "Symbolizing resilience in winter.", 
     "img": "suihan.jpg"},
    
    {"name_cn": "揽山亭", "name_en": "Lanshan Pavilion", 
     "lat": 31.390958 , "lon": 120.941150,
     "type_cn": "亭台", "type_en": "Pavilion", 
     "desc_cn": "登高望远。", "desc_en": "High vantage point.", 
     "img": "lanshan.jpg"},
    
    {"name_cn": "香雪", "name_en": "Xiangxue", 
     "lat": 31.391848, "lon": 120.940073,
     "type_cn": "景观", "type_en": "Scenic", 
     "desc_cn": "梅花盛开如雪。", "desc_en": "Plum blossoms like snow.", 
     "img": "xiangxue.jpg"},
    
    {"name_cn": "水云居", "name_en": "Shuiyunju", 
     "lat": 31.392246, "lon": 120.940403, 
     "type_cn": "茶室", "type_en": "Teahouse", 
     "desc_cn": "临水而居。", "desc_en": "Living by the water.", 
     "img": "shuiyunju.jpg"},
    
    {"name_cn": "猫空", "name_en": "Momomi Cafe", 
     "lat": 31.391582, "lon": 120.940302,
     "type_cn": "咖啡", "type_en": "Cafe", 
     "desc_cn": "书店与咖啡。", "desc_en": "Bookstore and cafe.", 
     "img": "maokong.jpg"},


  {
    "name_cn": "娄曲堂",
    "name_en": "Louqu Hall",
    "lat": 31.392028, "lon": 120.940924,
    "type_cn": "戏台",
    "type_en": "Stage",
    "desc_cn": "亭林园中类似戏台的亭子空间，曾用于传统表演与集会活动。",
    "desc_en": "A pavilion resembling a traditional opera stage, historically used for performances and gatherings.",
    "img": "louqu.jpg"
  },

  {
    "name_cn": "含晖亭",
    "name_en": "Han Hui Pavilion",
    "lat": 31.391863, "lon": 120.940962,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "取“含光纳晖”之意的观景亭，适合驻足休憩。",
    "desc_en": "A pavilion named for embracing light and glow, ideal for resting and viewing the surroundings.",
    "img": "hanhui.jpg"
  },

  {
    "name_cn": "山高水长",
    "name_en": "Mountains High, Waters Long Bridge",
    "lat": 31.391168, "lon": 120.942022,
    "type_cn": "桥",
    "type_en": "Bridge",
    "desc_cn": "连接亭林园主园与留晖园的重要桥梁，寓意精神绵延不绝。",
    "desc_en": "A bridge linking the main Tinglin Garden and Liuhui Garden, symbolizing enduring spirit and continuity.",
    "img": "shangaoshuichang.jpg"
  },

  {
    "name_cn": "好吃集市",
    "name_en": "Yummy Market",
    "lat":31.391557, "lon": 120.941787,
    "type_cn": "市集",
    "type_en": "Market",
    "desc_cn": "园内市集，提供烤串、煎饼、棉花糖等小吃，充满生活气息。",
    "desc_en": "A lively garden market offering skewers, pancakes, cotton candy, and other snacks.",
    "img": "haochi.jpg"
  },

  {
    "name_cn": "夕秀轩",
    "name_en": "Xixiu Pavilion",
    "lat": 31.392739, "lon": 120.941594,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "在傍晚光影中尤为秀美的建筑空间。",
    "desc_en": "A pavilion especially beautiful in the soft evening light.",
    "img": "xixiu.jpg"
  },

  {
    "name_cn": "荷香榭",
    "name_en": "Lotus Fragrance Pavilion",
    "lat": 31.392392, "lon": 120.941540,
    "type_cn": "水榭",
    "type_en": "Waterside Pavilion",
    "desc_cn": "临水而建，夏季荷花盛开时清香四溢。",
    "desc_en": "A waterside pavilion filled with lotus fragrance in summer.",
    "img": "hexiang.jpg"
  },

  {
    "name_cn": "玉犀双亭",
    "name_en": "Twin Jade Rhinoceros Pavilions",
    "lat": 31.394017, "lon": 120.947993,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "成对而立的亭子，因石刻与造型得名。",
    "desc_en": "A pair of pavilions named after jade-like stone features.",
    "img": "yuxi.jpg"
  },



  {
    "name_cn": "琼花王",
    "name_en": "King of Qionghua",
    "lat": 31.393330, "lon": 120.948924,
    "type_cn": "古树",
    "type_en": "Ancient Tree",
    "desc_cn": "亭林园著名古树，被誉为“琼花王”。",
    "desc_en": "A famous ancient tree known as the King of Qionghua.",
    "img": "qionghua_king.jpg"
  },

  {
    "name_cn": "戒石碑",
    "name_en": "Admonition Stone Stele",
    "lat": 31.393075, "lon": 120.948308,
    "type_cn": "碑刻",
    "type_en": "Stele",
    "desc_cn": "象征清廉与自省精神的石碑。",
    "desc_en": "A stone stele symbolizing integrity and self-discipline.",
    "img": "jieshi.jpg"
  },

  {
    "name_cn": "顾炎武石像",
    "name_en": "Gu Yanwu Statue",
    "lat": 31.393474, "lon": 120.948087,
    "type_cn": "雕塑",
    "type_en": "Statue",
    "desc_cn": "思想家顾炎武的石像，园内精神核心之一。",
    "desc_en": "A stone statue of Gu Yanwu, representing the spiritual core of the garden.",
    "img": "guyanwu_statue.jpg"
  },

  {
    "name_cn": "录竹居",
    "name_en": "Luzhu Courtyard",
    "lat": 31.392127,  "lon":120.946970,
    "type_cn": "庭院",
    "type_en": "Courtyard",
    "desc_cn": "以竹为主题的静谧庭院空间。",
    "desc_en": "A tranquil courtyard themed around bamboo.",
    "img": "luzhu_courtyard.jpg"
  },

  {
    "name_cn": "春风亭",
    "name_en": "Spring Breeze Pavilion",
    "lat": 31.392257, "lon": 120.946672,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "春季尤为宜人的观景亭。",
    "desc_en": "A pavilion best enjoyed during springtime breezes.",
    "img": "chunfeng.jpg"
  },

  {
    "name_cn": "丹实琼芳",
    "name_en": "Red Fruit & Jade Fragrance",
    "lat": 31.391831, "lon": 120.945908,
    "type_cn": "景点",
    "type_en": "Scenic Spot",
    "desc_cn": "由树木与奇石构成的园林景观组合。",
    "desc_en": "A scenic composition of trees and ornamental rocks.",
    "img": "danshiqiongfang.jpg"
  },

  {
    "name_cn": "养馀园",
    "name_en": "Yangyu Garden",
    "lat": 31.391675, "lon": 120.945031,
    "type_cn": "庭院 / 咖啡店",
    "type_en": "Courtyard / Café",
    "desc_cn": "拾级而上进入的庭院空间，内设咖啡店与盆栽陈设。",
    "desc_en": "A courtyard reached by steps, featuring a café and potted plants.",
    "img": "yangyu_garden.jpg"
  },

  {
    "name_cn": "牡丹亭",
    "name_en": "Peony Pavilion",
    "lat": 31.392805,  "lon": 120.946696,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "临水而建的传统园林亭阁，以《牡丹亭》意象命名，体现江南园林中诗意与戏曲文化的融合。",
    "desc_en": "A waterside pavilion inspired by the classical drama The Peony Pavilion, embodying the poetic and theatrical spirit of Jiangnan garden aesthetics.",
    "img": "mudan.jpg"
  },

  {
    "name_cn": "花雨轩",
    "name_en": "Flower Rain Pavilion",
    "lat": 31.391309, "lon": 120.944622,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "花木繁盛时如雨般飘落的观赏空间。",
    "desc_en": "A pavilion where falling petals resemble gentle rain.",
    "img": "huayu.jpg"
  },

  {
    "name_cn": "小西湖",
    "name_en": "Small West Lake",
    "lat":31.390602,  "lon":120.943483,
    "type_cn": "湖泊",
    "type_en": "Lake",
    "desc_cn": "‘小西湖’，园中特色水域景观之一，水面开阔宁静。",
    "desc_en": "A calm and open water feature within the garden. It's also called small 'West Lake'.",
    "img": "chuncai_lake.jpg"
  },

  {
    "name_cn": "爱莲亭",
    "name_en": "Lotus Appreciation Pavilion",
    "lat": 31.391593, "lon": 120.942543,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "体现“爱莲”品格的观景亭。",
    "desc_en": "A pavilion celebrating the virtue and beauty of lotus flowers.",
    "img": "ailian.jpg"
  },

  {
    "name_cn": "罗汉桥",
    "name_en": "Arhat Bridge",
    "lat": 31.391351, "lon": 120.942443,
    "type_cn": "桥",
    "type_en": "Bridge",
    "desc_cn": "园内造型古朴的桥梁景观。",
    "desc_en": "A traditional-style bridge within the garden.",
    "img": "luohan_bridge.jpg"
  },

  {
    "name_cn": "影山轩",
    "name_en": "Shadow Mountain Pavilion",
    "lat": 31.391419, "lon": 120.940895,
    "type_cn": "亭台",
    "type_en": "Pavilion",
    "desc_cn": "以走廊与光影变化见长的空间。",
    "desc_en": "A space known for its galleria forest and shifting light and shadow.",
    "img": "yingshan.jpg"
  },

    {
      "name_cn": "革命烈士陵园",
      "name_en": "Revolutionary Martyrs’ Cemetery",
      "lat": 31.395059, 
      "lon": 120.948594,
      "type_cn": "纪念地",
      "type_en": "Memorial Site",
      "desc_cn": "纪念近现代革命烈士的纪念空间，环境庄重肃穆，是昆山重要的爱国主义教育与历史记忆场所。",
      "desc_en": "A solemn memorial dedicated to revolutionary martyrs, serving as an important site of patriotic education and collective historical memory in Kunshan.",
      "img": "geming.jpg"
    },
    {
      "name_cn": "文笔峰",
      "name_en": "Wenbi Peak",
      "lat": 31.392501,
      "lon": 120.943750,
      "type_cn": "观景点",
      "type_en": "Scenic Viewpoint",
      "desc_cn": "玉峰山东峰，因山势挺拔如文笔得名。清代康熙帝曾登临题“鸢飞鱼跃”，为亭林园重要的人文地标。",
      "desc_en": "The eastern peak of Yufeng Mountain, named for its brush-like silhouette. Emperor Kangxi once visited and inscribed “Kites Soaring, Fish Leaping,” marking it as a key cultural landmark.",
      "img": "wenbi.jpg"
    },
    {
      "name_cn": "玉峰山（马鞍山）",
      "name_en": "Yufeng Mountain (Ma’an Hill)",
      "lat": 31.393130, 
      "lon": 120.944585,
      "type_cn": "自然景观",
      "type_en": "Natural Landmark",
      "desc_cn": "昆山境内唯一山体，海拔80.8米，因石色洁白如玉得名。山体由寒武纪硅质灰岩构成，形成玲珑石地貌。自南朝建慧聚寺，至清代康熙登临题字，园内汇集顾炎武纪念馆与昆石文化遗存，是自然与人文交织的核心象征。",
      "desc_en": "The only natural hill in Kunshan, rising 80.8 meters above the plain. Named for its jade-like white limestone, the mountain features distinctive Cambrian quartz-rich rock formations. With origins tracing back to the Liang Dynasty’s Huiju Temple and later imperial visits, it integrates natural geology with layered cultural heritage, including Gu Yanwu memorial sites and Kun stone history.",
      "img": "yufeng_mountain.jpg"
    },
    {
      "name_cn": "玉出昆冈",
      "name_en": "Jade from the Kun Ridge Archway",
      "lat": 31.392214, 
      "lon": 120.949087,
      "type_cn": "牌坊 / 文化象征",
      "type_en": "Archway / Cultural Symbol",
      "desc_cn": "亭林园东大门牌坊题“玉出昆冈”。“玉”既指玉峰山所产洁白玲珑的昆石（石英含量高达98%以上，被誉为“石中之玉”），亦喻昆山人杰地灵、贤才辈出。对联中“归奇顾怪”分别指归庄与顾炎武，象征此地物华天宝与人文鼎盛。",
      "desc_en": "The eastern entrance archway bears the inscription “Jade from the Kun Ridge.” The word “jade” refers both to the locally quarried Kun stone-renowned for its translucent, jade-like beauty—and metaphorically to the talented figures born in Kunshan. The couplet references Gui Zhuang and Gu Yanwu, celebrating both material richness and intellectual heritage.",
      "img": "yuchukungang.jpg"
    },
    {
      "name_cn": "亭林园西入口",
      "name_en": "Tinglin Garden West Entrance",
      "lat": 31.389261,
      "lon": 120.941908,
      "type_cn": "园林入口",
      "type_en": "Garden Entrance",
      "desc_cn": "西入口悬挂“亭林园”匾额，由此进入留晖园景区。空间由城市过渡至园林，形成视觉与情绪的缓冲，是游览动线的重要起点。",
      "desc_en": "Marked by the plaque “Tinglin Garden,” the west entrance leads visitors into the Liuhui Garden section. It serves as a spatial threshold, transitioning from urban context into classical garden scenery and shaping the visitor’s experiential sequence.",
      "img": "west_entrance.jpg"
    },
    {
      "name_cn": "妙峰塔",
      "name_en": "Miaofeng Pagoda",
      "lat": 31.393190, 
      "lon":120.945942,
      "type_cn": "古塔",
      "type_en": "Ancient Pagoda",
      "desc_cn": "仿木塔结构建造，设有腰檐与栏杆。塔身各层雕刻佛像，兼具宗教象征与精细石刻艺术，是玉峰山重要的历史遗存。",
      "desc_en": "A stone pagoda modeled after traditional wooden tower structures, complete with projecting eaves and railings. Each level features carved Buddha figures, combining religious symbolism with refined stone craftsmanship.",
      "img": "miaofeng.jpg"
    },

    {
      "name_cn": "一览亭",
      "name_en": "Yilan Pavilion",
      "lat": 31.393190, 
      "lon": 120.945369,
      "type_cn": "观景亭",
      "type_en": "Scenic Pavilion",
      "desc_cn": "位于玉峰山接近山顶，可俯瞰昆山城景。登亭远望，现代建筑与自然山水交织成景，展现城市发展与传统山林空间的对话。",
      "desc_en": "Located above midway up Yufeng Mountain, this pavilion offers panoramic views of Kunshan. From here, visitors observe the layered skyline where modern urban forms merge with natural hills and waterways.",
      "img": "yilan.jpg"
    }


]



In [75]:
from PIL import Image
from io import BytesIO
from PIL import ExifTags  #  ExifTags

def get_base64_image(image_filename):
    if not image_filename:
        return "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mP8/x8AAwMCAO+ip1sAAAAASUVORK5CYII=" 
    
    path = os.path.join(IMAGE_FOLDER, image_filename)
    
    # 路径匹配逻辑
    if not os.path.exists(path):
        if os.path.exists(IMAGE_FOLDER):
            folder_files = os.listdir(IMAGE_FOLDER)
            base = os.path.basename(path)
            for f in folder_files:
                if f.lower() == base.lower():
                    path = os.path.join(IMAGE_FOLDER, f)
                    break
    
    if not os.path.exists(path):
        print(f"警告: 找不到图片 {image_filename}")
        return "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mP8/x8AAwMCAO+ip1sAAAAASUVORK5CYII=" 

    try:
        with Image.open(path) as img:
            # === 新增：处理手机照片旋转 (Exif Orientation) ===
            try:
                for orientation in ExifTags.TAGS.keys():
                    if ExifTags.TAGS[orientation] == 'Orientation':
                        break
                exif = img._getexif()
                if exif is not None:
                    orientation = exif.get(orientation)
                    if orientation == 3:
                        img = img.rotate(180, expand=True)
                    elif orientation == 6:
                        img = img.rotate(270, expand=True)
                    elif orientation == 8:
                        img = img.rotate(90, expand=True)
            except (AttributeError, KeyError, IndexError):
                # 没 Exif 信息就不管
                pass
                
            # 转换为 RGB (防止 PNG 透明通道报错)
            if img.mode in ("RGBA", "P"):
                img = img.convert("RGB")
            
            # 压缩逻辑：如果宽度大于 400px，就等比缩小
            if img.width > 400:
                ratio = 400 / img.width
                new_height = int(img.height * ratio)
                img = img.resize((400, new_height), Image.Resampling.LANCZOS)
            
            # 保存到内存缓冲区
            buffered = BytesIO()
            # 降低质量保存为 JPEG，quality=70 足够清晰且体积很小
            img.save(buffered, format="JPEG", quality=70)
            
            return f"data:image/jpeg;base64,{base64.b64encode(buffered.getvalue()).decode('utf-8')}"
            
    except Exception as e:
        print(f"处理图片错误 {image_filename}: {e}")
        return "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mP8/x8AAwMCAO+ip1sAAAAASUVORK5CYII=" 


In [77]:
# ================= 3. 地图初始化 =================

CENTER_LAT = 31.392806
CENTER_LON = 120.944707

m = folium.Map(
    location=[CENTER_LAT, CENTER_LON],
    zoom_start=16, 
    tiles=None,    # 清除默认
    control_scale=True
)

# 添加 OpenStreetMap 图层
folium.TileLayer(
    tiles="https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
    attr='&copy; <a href="https://www.openstreetmap.org/#map=17/31.392284/120.943862" target="_blank">OpenStreetMap contributors</a>',
    name="OpenStreetMap"
).add_to(m)


In [78]:
print(f"正在处理 {len(locations)} 个地点...")

for loc in locations:
    # 读取数据
    name_cn = loc.get("name_cn", "未知地点")
    name_en = loc.get("name_en", "Unknown Spot")
    desc_cn = loc.get("desc_cn", "暂无描述").replace("'", "&#39;")
    desc_en = loc.get("desc_en", "No description available.").replace("'", "&#39;")
    type_cn = loc.get("type_cn", "景点")
    type_en = loc.get("type_en", "Spot")
    
    # 图片处理
    img_name = loc.get("img", "")
    img_src = get_base64_image(img_name)
    
    # 3. 视频按钮
    video_btn_html = ""
    if "video_link" in loc and loc["video_link"]:
        target_url = loc["video_link"]
        btn_text = "查看详情 / View Detail"
        
        video_btn_html = f"""
        <div style="margin-top: 15px;">
            <!-- 建议改回 target="_blank"，在服务器上这最稳妥 -->
            <a href="{target_url}" target="_blank" style="
                display: flex;
                align-items: center;
                justify-content: center;
                width: 100%;
                padding: 10px 0;
                background: linear-gradient(135deg, #2E8B57, #3CB371);
                color: white;
                text-decoration: none;
                border-radius: 8px;
                font-size: 14px;
                font-weight: bold;
                box-shadow: 0 4px 6px rgba(46, 139, 87, 0.3);
                transition: transform 0.2s;
            " onmouseover="this.style.transform='scale(1.02)'" 
              onmouseout="this.style.transform='scale(1)'">
                <span style="margin-right: 8px;">👉</span> 
                {btn_text}
            </a>
        </div>
        """




    # --- 4. 组装 Popup HTML ---
    html_content = f"""
    <div style="
        font-family: 'Noto Sans SC', 'Helvetica Neue', Arial, sans-serif;
        width: 320px;
        background: #ffffff;
        border-radius: 12px;
        overflow: hidden;
        box-shadow: 0 10px 25px rgba(0,0,0,0.2);
    ">
      <!-- 图片容器 -->
      <div style="
          position: relative; 
          height: 200px; 
          overflow: hidden; 
          background: #f0f0f0;
          display: flex;
          align-items: center;
          justify-content: center;
      ">
        <div style="
            position: absolute; top: -10%; left: -10%; width: 120%; height: 120%;
            background-image: url('{img_src}');
            background-size: cover;
            background-position: center;
            filter: blur(10px) brightness(0.8);
            z-index: 0;
        "></div>
        
        <img src="{img_src}" style="
            position: relative;
            z-index: 1;
            max-width: 100%; 
            max-height: 100%; 
            object-fit: contain; 
            box-shadow: 0 5px 15px rgba(0,0,0,0.3);
        " />
        
        <div style="
            position: absolute; top: 12px; right: 12px;
            background: rgba(255,255,255,0.95);
            padding: 4px 12px;
            border-radius: 20px;
            font-size: 11px;
            font-weight: bold;
            color: #2E8B57;
            box-shadow: 0 2px 4px rgba(0,0,0,0.15);
            z-index: 2;
        ">
            {type_cn} {type_en}
        </div>
      </div>
      
      <!-- 文字内容 -->
      <div style="padding: 20px;">
        <div style="margin-bottom: 12px; border-bottom: 1px solid #eee; padding-bottom: 10px;">
            <h3 style="margin: 0; font-size: 20px; color: #333; font-weight: 800;">{name_cn}</h3>
            <p style="margin: 4px 0 0; font-size: 13px; color: #888; font-style: italic;">{name_en}</p>
        </div>
        
        <div style="max-height: 150px; overflow-y: auto;">
            <p style="font-size: 13px; line-height: 1.6; color: #444; margin-bottom: 8px;">
                {desc_cn}
            </p>
            <p style="font-size: 12px; line-height: 1.5; color: #777; font-style: italic;">
                {desc_en}
            </p>
        </div>
        
        <!-- 这里插入按钮变量 -->
        {video_btn_html}
        
      </div>
    </div>
    """
    
    # 5. 添加 Marker
    # 直接把 html_content 塞给 Popup，不要 IFrame！
    # 你的 html_content 里已经写了 width: 320px，所以布局不会乱
    popup = folium.Popup(html_content, max_width=360)  # max_width 稍微给大一点点容错

    
    # 图标样式
    icon_html = f"""
    <div style="
        width: 36px; height: 36px; 
        border-radius: 50%; 
        border: 2px solid #ffffff;
        box-shadow: 0 4px 8px rgba(0,0,0,0.4);
        background-image: url('{img_src}');
        background-position: center center;
        background-size: cover;
        position: relative;
    "></div>
    <div style="
        position: absolute; bottom: -6px; left: 50%; transform: translateX(-50%);
        width: 0; height: 0; 
        border-left: 6px solid transparent;
        border-right: 6px solid transparent;
        border-top: 6px solid #ffffff;
    "></div>
    """
    
    folium.Marker(
        location=[loc["lat"], loc["lon"]], 
        popup=popup,
        tooltip=f"{name_cn} {name_en}",
        icon=DivIcon(icon_size=(40, 50), icon_anchor=(20, 50), html=icon_html)
    ).add_to(m)


output_file = "tinglin_map_fixed.html"
m.save(output_file)
print(f"成功！地图已生成并保存至: {output_file}")

正在处理 43 个地点...
成功！地图已生成并保存至: tinglin_map_fixed.html
